# 번개장터 + 네이버쇼핑 데이터 합쳐서 학습

In [ ]:
# GPU 에러 보기 위함

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

# 환경 설정

In [ ]:
# 통합코드 (GPT)

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip3 install kobert-transformers

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model, get_tokenizer

device = torch.device("cuda:0") # GPU
# device = torch.device("cpu") # CPU
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_kobert_model()
tokenizer = get_tokenizer() # vocab 역할

# 이후부터 모델 및 토크나이저를 사용하여 작업 진행.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
# device = torch.device("cuda:0") # GPU
# device = torch.device("cpu") # CPU

# 데이터셋 전처리

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
# data = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/nv_bj_one.csv')
data = pd.read_csv('/content/drive/MyDrive/data/nv_bj_one.csv')

In [ ]:
data

,id,name,cat_id
0,38241346937-310300400,쟈딕앤볼테르 Kayak 카키 아모르 카모 레터링 프린트 야상 자켓 점퍼,310300400
1,39945841940-310300400,카인 후드 바람막이 점퍼,310300400
2,40085020674-310300400,데이나믹 툴린스 고어텍스 여자 후드 윈드브레이커 자켓 20897733 1A00008...,310300400
3,38856225697-310300400,ZOOC 반팔 데님자켓 Z222PSF403 1544239,310300400
4,83356850408-310300400,봄 오버핏 힙합 점퍼 가을 데님 여성 재킷 야상 스트릿 아우터 여자 야구 학생,310300400
...,...,...,...
172495,152140476,백설공주키링,400600
172496,165755610,헬로키티 스트랩 키링 열쇠고리 고전 레진키링,400600
172497,103519257,가격내림 마르지엘라 키링 카드지갑 블랙,400600
172498,165256840,하이큐 카게야마 토비오 키링,400600


In [ ]:
LABEL2NUM = {
    '310300200':0,
    '310300100':1,
    '310300300':2,
    '310300400':3,
    '310300600':4,
    '310300500':5,
    '310260800':6,
    '310260700':7,
    '310260600':8,
    '310260500':9,
    '310260400':10,
    '310260100':11,
    '310260200':12,
    '310260300':13,
    '310150080':14,
    '310150010':15,
    '310150030':16,
    '310150090':17,
    '310150040':18,
    '310150070':19,
    '310130030':20,
    '310130080':21,
    '310130040':22,
    '310120030':23,
    '310120110':24,
    '310120020':25,
    '310250':26,
    '310400100':27,
    '310400200':28,
    '310200200':29,
    '310200100':30,
    '310220300':31,
    '310220100':32,
    '310220200':33,
    '320300300':34,
    '320300200':35,
    '320300100':36,
    '320300400':37,
    '320300600':38,
    '320300500':39,
    '320210600':40,
    '320210500':41,
    '320210700':42,
    '320210400':43,
    '320210100':44,
    '320210200':45,
    '320210300':46,
    '320120600':47,
    '320120100':48,
    '320120200':49,
    '320120300':50,
    '320120700':51,
    '320400':52,
    '320500100':53,
    '320500200':54,
    '320160700':55,
    '320160800':56,
    '320180700':57,
    '320180600':58,
    '405100':59,
    '405300300':60,
    '405300100':61,
    '405300200':62,
    '405200300':63,
    '405200400':64,
    '405200100':65,
    '405200200':66,
    '405400200':67,
    '405400300':68,
    '405400400':69,
    '405400500':70,
    '405400100':71,
    '405400600':72,
    '430100400':73,
    '430100100':74,
    '430100200':75,
    '430100300':76,
    '430100500':77,
    '430200100':78,
    '430200300':79,
    '430200200':80,
    '430200500':81,
    '430200400':82,
    '430300100':83,
    '430400':84,
    '430500':85,
    '421200100':86,
    '421200200':87,
    '421200300':88,
    '421200400':89,
    '421200500':90,
    '421100100':91,
    '421100200':92,
    '421100300':93,
    '421100400':94,
    '421100500':95,
    '421300100':96,
    '422200100':97,
    '422200200':98,
    '422200300':99,
    '422200400':100,
    '422200500':101,
    '422200600':102,
    '422200700':103,
    '422100100':104,
    '422100200':105,
    '422100300':106,
    '422100400':107,
    '422100500':108,
    '422100600':109,
    '422300100':110,
    '422300200':111,
    '422300300':112,
    '422400100':113,
    '422400200':114,
    '422500100':115,
    '422500200':116,
    '422500300':117,
    '422500400':118,
    '422500500':119,
    '422600100':120,
    '422600200':121,
    '400070100':122,
    '400070700':123,
    '400070200':124,
    '400070500':125,
    '400080300':126,
    '400080200':127,
    '400120100':128,
    '400120200':129,
    '400130100':130,
    '400130200':131,
    '400110100':132,
    '400110200':133,
    '400140100':134,
    '400140200':135,
    '400090':136,
    '400600':137
}
# cate_num

In [ ]:
LABEL2NAME = {
    '310300200':{'대분류':'여성의류', '중분류':'아우터', '소분류':'패딩'},
    '310300100':{'대분류':'여성의류', '중분류':'아우터', '소분류':'점퍼'},
    '310300300':{'대분류':'여성의류', '중분류':'아우터', '소분류':'코트'},
    '310300400':{'대분류':'여성의류', '중분류':'아우터', '소분류':'자켓'},
    '310300600':{'대분류':'여성의류', '중분류':'아우터', '소분류':'가디건'},
    '310300500':{'대분류':'여성의류', '중분류':'아우터', '소분류':'조끼/베스트'},
    '310260800':{'대분류':'여성의류', '중분류':'상의', '소분류':'니트/스웨터'},
    '310260700':{'대분류':'여성의류', '중분류':'상의', '소분류':'후드티/후드집업'},
    '310260600':{'대분류':'여성의류', '중분류':'상의', '소분류':'맨투맨'},
    '310260500':{'대분류':'여성의류', '중분류':'상의', '소분류':'블라우스'},
    '310260400':{'대분류':'여성의류', '중분류':'상의', '소분류':'셔츠'},
    '310260100':{'대분류':'여성의류', '중분류':'상의', '소분류':'반팔 티셔츠'},
    '310260200':{'대분류':'여성의류', '중분류':'상의', '소분류':'긴팔 티셔츠'},
    '310260300':{'대분류':'여성의류', '중분류':'상의', '소분류':'민소매 티셔츠'},
    '310150080':{'대분류':'여성의류', '중분류':'바지', '소분류':'데님/청바지'},
    '310150010':{'대분류':'여성의류', '중분류':'바지', '소분류':'슬랙스'},
    '310150030':{'대분류':'여성의류', '중분류':'바지', '소분류':'면바지'},
    '310150090':{'대분류':'여성의류', '중분류':'바지', '소분류':'반바지'},
    '310150040':{'대분류':'여성의류', '중분류':'바지', '소분류':'트레이닝/조거팬츠'},
    '310150070':{'대분류':'여성의류', '중분류':'바지', '소분류':'레깅스'},
    '310130030':{'대분류':'여성의류', '중분류':'치마', '소분류':'롱 스커트'},
    '310130080':{'대분류':'여성의류', '중분류':'치마', '소분류':'미디 스커트'},
    '310130040':{'대분류':'여성의류', '중분류':'치마', '소분류':'미니 스커트'},
    '310120030':{'대분류':'여성의류', '중분류':'원피스', '소분류':'롱 원피스'},
    '310120110':{'대분류':'여성의류', '중분류':'원피스', '소분류':'미디 원피스'},
    '310120020':{'대분류':'여성의류', '중분류':'원피스', '소분류':'미니 원피스'},
    '310250':{'대분류':'여성의류', '중분류':'점프수트'},
    '310400100':{'대분류':'여성의류', '중분류':'셋업/세트', '소분류':'정장/셋업'},
    '310400200':{'대분류':'여성의류', '중분류':'셋업/세트', '소분류':'트레이닝/스웨트 셋업'},
    '310200200':{'대분류':'여성의류', '중분류':'언더웨어/홈웨어', '소분류':'홈웨어'},
    '310200100':{'대분류':'여성의류', '중분류':'언더웨어/홈웨어', '소분류':'언더웨어'},
    '310220300':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'코스튬/코스프레'},
    '310220100':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'한복'},
    '310220200':{'대분류':'여성의류', '중분류':'테마/이벤트', '소분류':'드레스'},
    '320300300':{'대분류':'남성의류', '중분류':'아우터', '소분류':'패딩'},
    '320300200':{'대분류':'남성의류', '중분류':'아우터', '소분류':'점퍼'},
    '320300100':{'대분류':'남성의류', '중분류':'아우터', '소분류':'코트'},
    '320300400':{'대분류':'남성의류', '중분류':'아우터', '소분류':'자켓'},
    '320300600':{'대분류':'남성의류', '중분류':'아우터', '소분류':'가디건'},
    '320300500':{'대분류':'남성의류', '중분류':'아우터', '소분류':'조끼/베스트'},
    '320210600':{'대분류':'남성의류', '중분류':'상의', '소분류':'후드티/후드집업'},
    '320210500':{'대분류':'남성의류', '중분류':'상의', '소분류':'맨투맨'},
    '320210700':{'대분류':'남성의류', '중분류':'상의', '소분류':'니트/스웨터'},
    '320210400':{'대분류':'남성의류', '중분류':'상의', '소분류':'셔츠'},
    '320210100':{'대분류':'남성의류', '중분류':'상의', '소분류':'반팔 티셔츠'},
    '320210200':{'대분류':'남성의류', '중분류':'상의', '소분류':'긴팔 티셔츠'},
    '320210300':{'대분류':'남성의류', '중분류':'상의', '소분류':'민소매 티셔츠'},
    '320120600':{'대분류':'남성의류', '중분류':'바지', '소분류':'데님/청바지'},
    '320120100':{'대분류':'남성의류', '중분류':'바지', '소분류':'면바지'},
    '320120200':{'대분류':'남성의류', '중분류':'바지', '소분류':'슬랙스'},
    '320120300':{'대분류':'남성의류', '중분류':'바지', '소분류':'트레이닝/조거팬츠'},
    '320120700':{'대분류':'남성의류', '중분류':'바지', '소분류':'반바지'},
    '320400':{'대분류':'남성의류', '중분류':'점프수트'},
    '320500100':{'대분류':'남성의류', '중분류':'셋업/세트', '소분류':'정장/셋업'},
    '320500200':{'대분류':'남성의류', '중분류':'셋업/세트', '소분류':'트레이닝/스웨트 셋업'},
    '320160700':{'대분류':'남성의류', '중분류':'언더웨어/홈웨어', '소분류':'언더웨어'},
    '320160800':{'대분류':'남성의류', '중분류':'언더웨어/홈웨어', '소분류':'홈웨어'},
    '320180700':{'대분류':'남성의류', '중분류':'테마/이벤트', '소분류':'코스튬/코스프레'},
    '320180600':{'대분류':'남성의류', '중분류':'테마/이벤트', '소분류':'한복'},
    '405100':{'대분류':'신발', '중분류':'스니커즈'},
    '405300300':{'대분류':'신발', '중분류':'남성화', '소분류':'샌들/슬리퍼'},
    '405300100':{'대분류':'신발', '중분류':'남성화', '소분류':'구두/로퍼'},
    '405300200':{'대분류':'신발', '중분류':'남성화', '소분류':'워커/부츠'},
    '405200300':{'대분류':'신발', '중분류':'여성화', '소분류':'샌들/슬리퍼'},
    '405200400':{'대분류':'신발', '중분류':'여성화', '소분류':'구두'},
    '405200100':{'대분류':'신발', '중분류':'여성화', '소분류':'단화/플랫슈즈'},
    '405200200':{'대분류':'신발', '중분류':'여성화', '소분류':'워커/부츠'},
    '405400200':{'대분류':'신발', '중분류':'스포츠화', '소분류':'농구화'},
    '405400300':{'대분류':'신발', '중분류':'스포츠화', '소분류':'골프화'},
    '405400400':{'대분류':'신발', '중분류':'스포츠화', '소분류':'축구/풋살화'},
    '405400500':{'대분류':'신발', '중분류':'스포츠화', '소분류':'테니스화'},
    '405400100':{'대분류':'신발', '중분류':'스포츠화', '소분류':'등산화/트레킹화'},
    '405400600':{'대분류':'신발', '중분류':'스포츠화', '소분류':'야구화'},
    '430100400':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'클러치백'},
    '430100100':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'숄더백'},
    '430100200':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'크로스백'},
    '430100300':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'토트백'},
    '430100500':{'대분류':'가방/지갑', '중분류':'여성가방', '소분류':'백팩'},
    '430200100':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'클러치'},
    '430200300':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'숄더백'},
    '430200200':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'크로스백'},
    '430200500':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'브리프케이스'},
    '430200400':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'백팩'},
    '430300100':{'대분류':'가방/지갑', '중분류':'남성가방', '소분류':'캐리어'},
    '430400':{'대분류':'가방/지갑', '중분류':'여성지갑'},
    '430500':{'대분류':'가방/지갑', '중분류':'남성지갑'},
    '421200100':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(디지털)'},
    '421200200':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(쿼츠)'},
    '421200300':{'대분류':'시계', '중분류':'남성시계', '소분류':'프리미엄 시계(오토매틱)'},
    '421200400':{'대분류':'시계', '중분류':'남성시계', '소분류':'일반 시계(메탈 밴드)'},
    '421200500':{'대분류':'시계', '중분류':'남성시계', '소분류':'일반 시계(가죽 밴드)'},
    '421100100':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(디지털)'},
    '421100200':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(쿼츠)'},
    '421100300':{'대분류':'시계', '중분류':'여성시계', '소분류':'프리미엄 시계(오토매틱)'},
    '421100400':{'대분류':'시계', '중분류':'여성시계', '소분류':'일반 시계(메탈 밴드)'},
    '421100500':{'대분류':'시계', '중분류':'여성시계', '소분류':'일반 시계(가죽 밴드)'},
    '421300100':{'대분류':'시계', '중분류':'시계용품', '소분류':'스트랩'},
    '422200100':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'다이아몬드 귀걸이'},
    '422200200':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'금 귀걸이'},
    '422200300':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'은 귀걸이'},
    '422200400':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'진주/유색보석 귀걸이'},
    '422200500':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'패션 귀걸이'},
    '422200600':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'피어싱'},
    '422200700':{'대분류':'쥬얼리', '중분류':'귀걸이/피어싱', '소분류':'귀찌/이어커프'},
    '422100100':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'다이아몬드 목걸이'},
    '422100200':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'금 목걸이'},
    '422100300':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'은 목걸이'},
    '422100400':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'진주/유색보석 목걸이'},
    '422100500':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'패션 목걸이'},
    '422100600':{'대분류':'쥬얼리', '중분류':'목걸이/펜던트', '소분류':'초커'},
    '422300100':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'금팔찌'},
    '422300200':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'은팔찌'},
    '422300300':{'대분류':'쥬얼리', '중분류':'팔찌', '소분류':'패션 팔찌'},
    '422400100':{'대분류':'쥬얼리', '중분류':'발찌', '소분류':'금발찌'},
    '422400200':{'대분류':'쥬얼리', '중분류':'발찌', '소분류':'패션발찌'},
    '422500100':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'다이아몬드 반지'},
    '422500200':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'금반지'},
    '422500300':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'은반지'},
    '422500400':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'진주/유색보석 반지'},
    '422500500':{'대분류':'쥬얼리', '중분류':'반지', '소분류':'패션반지'},
    '422600100':{'대분류':'쥬얼리', '중분류':'쥬얼리 세트', '소분류':'귀금속 쥬얼리 세트'},
    '422600200':{'대분류':'쥬얼리', '중분류':'쥬얼리 세트', '소분류':'패션 쥬얼리 세트'},
    '400070100':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'볼캡'},
    '400070700':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'버킷'},
    '400070200':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'스냅백'},
    '400070500':{'대분류':'패션 액세서리', '중분류':'모자', '소분류':'비니'},
    '400080300':{'대분류':'패션 액세서리', '중분류':'안경/선글라스', '소분류':'선글라스'},
    '400080200':{'대분류':'패션 액세서리', '중분류':'안경/선글라스', '소분류':'안경'},
    '400120100':{'대분류':'패션 액세서리', '중분류':'목도리/장갑', '소분류':'목도리'},
    '400120200':{'대분류':'패션 액세서리', '중분류':'목도리/장갑', '소분류':'장갑'},
    '400130100':{'대분류':'패션 액세서리', '중분류':'스카프/넥타이', '소분류':'스카프'},
    '400130200':{'대분류':'패션 액세서리', '중분류':'스카프/넥타이', '소분류':'넥타이'},
    '400110100':{'대분류':'패션 액세서리', '중분류':'벨트', '소분류':'남성 벨트'},
    '400110200':{'대분류':'패션 액세서리', '중분류':'벨트', '소분류':'여성 벨트'},
    '400140100':{'대분류':'패션 액세서리', '중분류':'양말/스타킹', '소분류':'양말'},
    '400140200':{'대분류':'패션 액세서리', '중분류':'양말/스타킹', '소분류':'스타킹'},
    '400090':{'대분류':'패션 액세서리', '중분류':'우산/양산'},
    '400600':{'대분류':'패션 액세서리', '중분류':'키링/키케이스'}
}
# cate_dict
# 라벨인코딩한 카테고리id와 카테고리명 저장해놓은 딕셔너리

In [ ]:
data['cate_enco'] = 0
for idx, row in data.iterrows():
    label_name = LABEL2NUM[str(row['cat_id'])]
    data.iloc[idx, 3] = label_name
data = data.drop(['cat_id'], axis=1)
data

,id,name,cate_enco
0,38241346937-310300400,쟈딕앤볼테르 Kayak 카키 아모르 카모 레터링 프린트 야상 자켓 점퍼,3
1,39945841940-310300400,카인 후드 바람막이 점퍼,3
2,40085020674-310300400,데이나믹 툴린스 고어텍스 여자 후드 윈드브레이커 자켓 20897733 1A00008...,3
3,38856225697-310300400,ZOOC 반팔 데님자켓 Z222PSF403 1544239,3
4,83356850408-310300400,봄 오버핏 힙합 점퍼 가을 데님 여성 재킷 야상 스트릿 아우터 여자 야구 학생,3
...,...,...,...
172495,152140476,백설공주키링,137
172496,165755610,헬로키티 스트랩 키링 열쇠고리 고전 레진키링,137
172497,103519257,가격내림 마르지엘라 키링 카드지갑 블랙,137
172498,165256840,하이큐 카게야마 토비오 키링,137


In [ ]:
len(data['cate_enco'].unique())

138

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172500 entries, 0 to 172499
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         172500 non-null  object
 1   name       172500 non-null  object
 2   cate_enco  172500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.9+ MB


In [ ]:
data_list = []
for q, label in zip(data['name'], data['cate_enco']):
    data1 = []
    data1.append(q)
    data1.append(label)

    data_list.append(data1)

In [ ]:
data_list[0] # 현재 코드

['쟈딕앤볼테르 Kayak 카키 아모르 카모 레터링 프린트 야상 자켓 점퍼', 3]

# Train data & Test data

In [ ]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=25, stratify=[data[1] for data in data_list])

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

138000
34500


# KoBERT 입력 데이터로 만들기

In [ ]:
from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, valid_length, label)

    def __len__(self):
        return len(self.labels)


In [ ]:
# Setting parameters
max_len = 64
batch_size = 48
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
num_classes = 138

In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
data_train[0]

(tensor([   2, 3758, 5947, 6812, 7436, 7565, 7697, 7612, 3307, 2195, 6149, 7096,
         4074, 7706,  671,  305,  274,  311,  360,  316,  142,  130,  172,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(3))

In [ ]:
data_test[0]

(tensor([   2, 2872, 7140, 6629, 7140,  517, 6438, 6037, 6765, 2044, 7495, 2095,
         6664,  586, 7119, 7483, 4257, 5808, 7921, 4643, 6122, 6855,  693,  342,
          188,  137,  140,    3,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(83))

In [ ]:
# 이전 코드로 확인
dataset_train[0] # [CLS], "화이트", "##무드", "중청", "##바", "##지", [SEP]

['화이트무드 중청바지', 310140030]

In [ ]:
dataset_train[0]

['인디안체크패턴 여름 바람막이 점퍼 MITCLYM320115', 3]

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# KoBERT 학습모델 만들기

In [ ]:
len(data['cate_enco'].unique()) # 카테고리 수 138개

138

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 138,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output


In [ ]:
# GPU

from torch.optim import AdamW

# BERT 모델 불러오기
kobert_model = get_kobert_model()
model = BERTClassifier(kobert_model, num_classes=num_classes, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss().to(device)

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
## CPU

# BERT 모델 불러오기
# kobert_model = get_kobert_model()
# model = BERTClassifier(kobert_model,  dr_rate=0.5)
# # optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss()

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# # 정확도 측정을 위한 함수 정의
# def calc_accuracy(X, Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices.cpu() == Y.cpu()).sum().data.numpy() / max_indices.size()[0]
#     return train_acc


# KoBERT 모델 학습시키기

In [ ]:
for i, (data1, data2, data3, data4) in enumerate(train_dataloader):
  print(data1)
  print(data2)
  print(data3)
  print(data4)
  break

tensor([[   2, 3758, 5947,  ...,    1,    1,    1],
        [   2, 1176, 4737,  ...,    1,    1,    1],
        [   2, 3080, 7318,  ...,    1,    1,    1],
        ...,
        [   2, 1727, 3886,  ...,    1,    1,    1],
        [   2,  581,  285,  ...,    1,    1,    1],
        [   2,  546, 7119,  ...,    1,    1,    1]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64])
tensor([  3,  77, 117,  64,  44,  76, 132, 129,  27,  79, 112, 112,  61, 107,
        111, 110, 103,  60, 124,  93,  46,  50, 123, 124,  82,  57,  83, 106,
        130, 123,  62,  77, 117,  74,  73,  19,   3,  19, 112,  32,  44,  20,
     

In [ ]:
# 에폭10으로 저장완료 ~~~~ 4시간 36분.

from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)
        out = model(token_ids, attention_mask)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask)

        # 상위 세 개의 카테고리를 추출하는 로직 추가
        probabilities = F.softmax(out, dim=1)
        _, top_indices = torch.topk(probabilities, k=3, dim=1)


        test_acc += calc_accuracy(top_indices , label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

# 모델 저장 경로 지정
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model.pth"

# 모델 저장
torch.save(model.state_dict(), save_path)

  0%|          | 0/2875 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.031569957733154 train acc 0.0
epoch 1 batch id 201 loss 4.940618515014648 train acc 0.005286069651741294
epoch 1 batch id 401 loss 4.9214863777160645 train acc 0.006961762261014132
epoch 1 batch id 601 loss 4.76765251159668 train acc 0.00967138103161398
epoch 1 batch id 801 loss 4.453807830810547 train acc 0.02413649604660841
epoch 1 batch id 1001 loss 3.9322335720062256 train acc 0.059482184482184494
epoch 1 batch id 1201 loss 3.326606512069702 train acc 0.10232792117679715
epoch 1 batch id 1401 loss 2.7724716663360596 train acc 0.1412830121341899
epoch 1 batch id 1601 loss 2.3998374938964844 train acc 0.17453935040599636
epoch 1 batch id 1801 loss 1.7755879163742065 train acc 0.2060776420507125
epoch 1 batch id 2001 loss 1.884508728981018 train acc 0.2343723971347657
epoch 1 batch id 2201 loss 1.6685094833374023 train acc 0.2601185067393609
epoch 1 batch id 2401 loss 1.6376527547836304 train acc 0.28502880744134385
epoch 1 batch id 2601 loss 1.55922424793243

  0%|          | 0/719 [00:00<?, ?it/s]

epoch 1 test acc 0.6898180343069075


  0%|          | 0/2875 [00:00<?, ?it/s]

...

epoch 9 test acc 0.8250753361149747


  0%|          | 0/2875 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.18909412622451782 train acc 0.9583333333333334
epoch 10 batch id 201 loss 0.11767102032899857 train acc 0.9547056384742957
epoch 10 batch id 401 loss 0.09254646301269531 train acc 0.9519430590191199
epoch 10 batch id 601 loss 0.6101337671279907 train acc 0.9524750415973391
epoch 10 batch id 801 loss 0.3317231237888336 train acc 0.9520911360799001
epoch 10 batch id 1001 loss 0.10421759635210037 train acc 0.9526931401931397
epoch 10 batch id 1201 loss 0.03654000908136368 train acc 0.9527824035525951
epoch 10 batch id 1401 loss 0.06247381493449211 train acc 0.9533369022127055
epoch 10 batch id 1601 loss 0.0825338289141655 train acc 0.9533754944826164
epoch 10 batch id 1801 loss 0.025367310270667076 train acc 0.9533592448639658
epoch 10 batch id 2001 loss 0.04059969261288643 train acc 0.9535232383808113
epoch 10 batch id 2201 loss 0.1166941449046135 train acc 0.9533450704225367
epoch 10 batch id 2401 loss 0.19692794978618622 train acc 0.9531271692350421
epoch 10 

  0%|          | 0/719 [00:00<?, ?it/s]

epoch 10 test acc 0.8254423582135691


In [ ]:
# 저장된 모델 불러오기

## GPU
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model1_kobert10.pth"
model.load_state_dict(torch.load(save_path))


## CPU
# device = torch.device("cpu")
# model = BERTClassifier(kobert_model, dr_rate=0.5)
# save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model_kobert_epoch5.pth"
# model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))


<All keys matched successfully>

# 새로운 문장 테스트

In [ ]:
# 새 코드

import numpy as np
from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
tok = get_tokenizer()

def predict(predict_sentence):
    category_id = 0  # 예측할 카테고리의 인덱스 값
    data = [(predict_sentence, category_id)]  # 상품명, 카테고리id
    dataset_another = BERTDataset(data, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)
    test_dataloader = torch.utils.data.DataLoader(dataset_another, batch_size=1, num_workers=0)  # 배치 크기를 1로 설정

    model.eval()

    for batch_id, (input_ids, attention_mask, valid_length, label) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        out = model(input_ids, attention_mask)

        # test_eval = []
        # for i in out:
        #     logits = i
        #     logits = logits.detach().cpu().numpy()

        #     if np.argmax(logits) in LABEL2NAME:
        #         category = LABEL2NAME[np.argmax(logits)]
        #         test_eval.append(category)

        logits = out.detach().cpu().numpy()  # 모델 출력을 NumPy 배열로 변환
        top_3_indices = np.argsort(logits, axis=1)[0, -3:]  # 가장 높은 확률을 가진 상위 세 개의 인덱스 선택

        test_eval = [LABEL2NAME[i] for i in top_3_indices]  # 인덱스에 해당하는 카테고리 이름 가져오기

        print(">> 입력하신 상품명으로부터 예측된 카테고리:")
        for category in test_eval:
            print(category)

        # if test_eval:
        #     print(">> 입력하신 상품명으로부터 예측된 카테고리:", test_eval[0])
        # else:
        #     print(">> 카테고리를 예측할 수 없습니다.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


# 학습1에서 입력해보기!!

In [ ]:
# 질문 무한 반복! 0 입력시 종료
while True:
    sentence = input("상품명을 입력해주세요 : ")
    if sentence == '0':
        break
    predict(sentence)
    print("\n")

상품명을 입력해주세요 : 귀염뽀짝 길다란 시계


KeyError: ignored